In [2]:
import pandas as pd

In [28]:
from scipy.stats import zscore
from sklearn.model_selection import train_test_split 

In [38]:
from sklearn.preprocessing import LabelEncoder

In [86]:
import numpy as np

In [259]:
df = pd.read_csv("Churn_Modelling.csv")

In [260]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [261]:
df.shape

(10000, 14)

In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [263]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [264]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


# Drop the columns which are unique for all users like IDs

In [265]:
df_new = df.drop(['RowNumber',"Surname","CustomerId"], axis=1)
df_new

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [266]:
le = LabelEncoder()
categorical_feature_mask = df_new.dtypes==object
categorical_cols = df_new.columns[categorical_feature_mask].tolist()
df_new[categorical_cols] = df_new[categorical_cols].apply(lambda col: le.fit_transform(col))
print(df_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  int32  
 2   Gender           10000 non-null  int32  
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int32(2), int64(7)
memory usage: 781.4 KB
None


In [300]:
df_new

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


# Distinguish the feature and target set

In [267]:
df_scaled = df_new.apply(zscore)
X_columns =  df_scaled.columns.tolist()[1:10]
Y_Columns = df.columns.tolist()[-1:]

X = df_scaled[X_columns].values
y = np.array(df_new['Exited']) # Exited

print(y)
print(X)

[1 0 1 ... 1 1 0]
[[-0.90188624 -1.09598752  0.29351742 ...  0.64609167  0.97024255
   0.02188649]
 [ 1.51506738 -1.09598752  0.19816383 ... -1.54776799  0.97024255
   0.21653375]
 [-0.90188624 -1.09598752  0.29351742 ...  0.64609167 -1.03067011
   0.2406869 ]
 ...
 [-0.90188624 -1.09598752 -0.27860412 ... -1.54776799  0.97024255
  -1.00864308]
 [ 0.30659057  0.91241915  0.29351742 ...  0.64609167 -1.03067011
  -0.12523071]
 [-0.90188624 -1.09598752 -1.04143285 ...  0.64609167 -1.03067011
  -1.07636976]]


# Divide the data set into training and test sets

In [271]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state= 8)

In [298]:
X_train

array([[-0.32660661,  0.33042097,  0.41707339, ...,  0.23397387,
         0.35136098,  0.45409568],
       [ 0.41088162, -0.29722845, -0.04969698, ..., -0.41974993,
         0.2631268 , -0.26355289],
       [-0.26269006, -0.31922543, -0.30333543, ..., -0.45081435,
         0.28260002, -0.48071808],
       ...,
       [ 0.52973741,  0.31902381,  0.06928721, ...,  0.22590344,
        -0.3603698 ,  0.19493733],
       [ 0.11730923,  0.34911441, -0.17957026, ..., -0.59221479,
        -0.39436019, -0.28109801],
       [ 0.09429018,  0.28060932, -0.05635766, ..., -0.47600724,
         0.29839258,  0.53266892]])

In [299]:
X_test

array([[ 0.11961429, -0.42759232,  0.63533595, ...,  0.25206841,
         0.37853375, -0.13253872],
       [ 0.10480879, -0.3746662 ,  0.58929281, ...,  0.22086812,
         0.33167995,  0.1222605 ],
       [ 0.50531507, -0.36554085, -0.15652774, ...,  0.21548867,
        -0.34375576,  0.41137911],
       ...,
       [ 0.11278809,  0.33565943, -0.2077281 , ...,  0.23768326,
        -0.37916142,  0.59518817],
       [-0.3643766 ,  0.36863206,  0.08006139, ...,  0.26103146,
        -0.41640736,  0.22923692],
       [-0.30802696,  0.31162433, -0.09515344, ...,  0.22066381,
        -0.35201134,  0.35994045]])

In [272]:
#Encoding the output class label (One-Hot Encoding)
y_train=to_categorical(y_train,2)
y_test=to_categorical(y_test,2)

In [242]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std_ = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Normalize the train and test data

In [273]:
from sklearn.preprocessing import Normalizer
normalize=Normalizer(norm="l2")
X_train=normalize.transform(X_train)

print(X_train)

[[-0.32660661  0.33042097  0.41707339 ...  0.23397387  0.35136098
   0.45409568]
 [ 0.41088162 -0.29722845 -0.04969698 ... -0.41974993  0.2631268
  -0.26355289]
 [-0.26269006 -0.31922543 -0.30333543 ... -0.45081435  0.28260002
  -0.48071808]
 ...
 [ 0.52973741  0.31902381  0.06928721 ...  0.22590344 -0.3603698
   0.19493733]
 [ 0.11730923  0.34911441 -0.17957026 ... -0.59221479 -0.39436019
  -0.28109801]
 [ 0.09429018  0.28060932 -0.05635766 ... -0.47600724  0.29839258
   0.53266892]]


In [274]:
X_test=normalize.transform(X_test)
print(X_test)

[[ 0.11961429 -0.42759232  0.63533595 ...  0.25206841  0.37853375
  -0.13253872]
 [ 0.10480879 -0.3746662   0.58929281 ...  0.22086812  0.33167995
   0.1222605 ]
 [ 0.50531507 -0.36554085 -0.15652774 ...  0.21548867 -0.34375576
   0.41137911]
 ...
 [ 0.11278809  0.33565943 -0.2077281  ...  0.23768326 -0.37916142
   0.59518817]
 [-0.3643766   0.36863206  0.08006139 ...  0.26103146 -0.41640736
   0.22923692]
 [-0.30802696  0.31162433 -0.09515344 ...  0.22066381 -0.35201134
   0.35994045]]


# Initialize & build the model.

In [249]:
from tensorflow.keras import Sequential

In [250]:
from tensorflow.keras.layers import Dense

In [301]:
classifier = Sequential()
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dense(20, activation = 'relu'))

classifier.add(Dense(2, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 45, epochs = 200,  verbose=1)



Epoch 1/200
178/178 [==============================] - 0s 2ms/step - loss: 0.5616 - accuracy: 0.7859
Epoch 2/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4687 - accuracy: 0.7970
Epoch 3/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4416 - accuracy: 0.7970
Epoch 4/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4284 - accuracy: 0.8121
Epoch 5/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4215 - accuracy: 0.8250
Epoch 6/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4172 - accuracy: 0.8279
Epoch 7/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8299
Epoch 8/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.8317
Epoch 9/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8332
Epoch 10/200
178/178 [==============================] - 0s 2ms/step - loss: 0.4052 - accura

178/178 [==============================] - 0s 2ms/step - loss: 0.3762 - accuracy: 0.8438
Epoch 79/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3761 - accuracy: 0.8416
Epoch 80/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3754 - accuracy: 0.8436
Epoch 81/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3755 - accuracy: 0.8436
Epoch 82/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3752 - accuracy: 0.8447
Epoch 83/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3747 - accuracy: 0.8435
Epoch 84/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3746 - accuracy: 0.8454
Epoch 85/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3748 - accuracy: 0.8450
Epoch 86/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3741 - accuracy: 0.8439
Epoch 87/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3739 - accuracy: 

178/178 [==============================] - 0s 2ms/step - loss: 0.3410 - accuracy: 0.8590
Epoch 158/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8576
Epoch 159/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8584
Epoch 160/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3403 - accuracy: 0.8586
Epoch 161/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3403 - accuracy: 0.8577
Epoch 162/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3409 - accuracy: 0.8583
Epoch 163/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3402 - accuracy: 0.8593
Epoch 164/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3400 - accuracy: 0.8580
Epoch 165/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3398 - accuracy: 0.8589
Epoch 166/200
178/178 [==============================] - 0s 2ms/step - loss: 0.3399 - a

In [302]:
classifier.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 6)                 60        
_________________________________________________________________
dense_52 (Dense)             (None, 20)                140       
_________________________________________________________________
dense_53 (Dense)             (None, 2)                 42        
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________


In [303]:
score = classifier.evaluate(X_test, y_test,verbose=1)

print(score)

63/63 [==============================] - 0s 1ms/step - loss: 0.3465 - accuracy: 0.8610
[0.3465089499950409, 0.8610000014305115]


# Predict the results

In [304]:
y_pred = classifier.predict(X_test)

In [305]:
y_pred = (y_pred > 0.5)

In [306]:
from sklearn.metrics import confusion_matrix
confmatrx = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

# Print the Accuracy score and confusion matrix

In [307]:
confmatrx

array([[1525,   62],
       [ 216,  197]], dtype=int64)

In [308]:
print (((confmatrx[0][0]+confmatrx[1][1])*100)/(len(y_test)), '% of testing data was classified correctly')

86.1 % of testing data was classified correctly


In [309]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.861